In [7]:
from songs import *

In [8]:
from numpy import random

In [9]:
MEMORY_TIMES = 64
STARTER = 8

ADDITIONAL_MEMORY = 8

MEMORY_SIZE = MEMORY_TIMES * 13 + ADDITIONAL_MEMORY

In [10]:
NAMES = []
for i in range(8):
    NAMES.append("[tb" + str(i) + "]")
for t in range(-MEMORY_TIMES, 0):
    for note_name in ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B", "C^"]:
        NAMES.append(note_name + str(t))

In [11]:
class Func:
    def __init__(self, Random):
        self.operations = ["conj"]
        self.operands = np.zeros((1, MEMORY_SIZE))
        self.support_variables = [[]]
        self.threshold = [0]
        self.final_decision_operation = 0
        self.Random = Random
        
    def apply(self, x):
        ans = 1
        self.final_decision_operation = -1
        
        vals = self.operands.dot(x) >= self.threshold
        
        for index, oper, val in zip(range(len(self.operations)), self.operations, vals):
            if oper == "conj" and val:
                ans = 0
                self.final_decision_operation = index
            if oper == "disj" and val:
                ans = 1
                self.final_decision_operation = index
        return ans
    
    def reward(self, memory):   
        self.support_variables[self.final_decision_operation].append(memory)
    
    def add(self, operation, memory, where_to_put):                      
        t = 1
        for i in range(where_to_put):
            if self.operations[i] != operation:
                for support_point in self.support_variables[i]:
                    t = max(t, support_point.dot(memory) + 1)          
        
        self.operations.insert(where_to_put, operation)
        self.support_variables.insert(where_to_put, [memory])        
        self.operands = np.vstack([self.operands[:where_to_put], memory, self.operands[where_to_put:]])
                                  
        self.threshold.insert(where_to_put, t)
    
    def print(self):
        print("0", end="")
        for oper, new_operand, t in zip(self.operations[1:], self.operands[1:], self.threshold[1:]):
            if oper == "conj":
                print(" ∧ not", end="")
            if oper == "disj":
                print(" ∨ ", end="")
            for i in range(MEMORY_SIZE):
                if new_operand[i]:
                    print(NAMES[i], end="")
            print("(", t, ")")
        print("")

In [12]:
class Player:
    def __init__(self):
        self.Random = np.random.RandomState(seed=179)
        self.play_note = [Func(self.Random) for i in range(13)]
        
    def play(self, memory):
        return np.array([self.play_note[i].apply(memory) for i in range(13)], dtype=int)
    
    def learn_to_play(self, song_to_learn, verbose=False):
        memory = np.zeros((MEMORY_SIZE), dtype=int)
        memory[-STARTER*13:] = song_to_learn.notes[:STARTER].flatten()

        t = STARTER
        memory[t % 8] = 1
        errors = 0

        while t < len(song_to_learn.notes):    
            output = self.play(memory)
            
            if verbose:
                print(output)

            for note in range(13):    
                if output[note] != song_to_learn.notes[t][note] or self.play_note[note].final_decision_operation == 0:
                    if song_to_learn.notes[t][note] == 1:
                        errors += 1
                        self.play_note[note].add("disj", memory, self.play_note[note].final_decision_operation + 1)
                        if verbose:
                            print("note " + str(note) + " should be pushed!")
                    if song_to_learn.notes[t][note] == 0:
                        errors += 1
                        self.play_note[note].add("conj", memory, self.play_note[note].final_decision_operation + 1)
                        if verbose:                            
                            print("note " + str(note) + " should not be pushed!")
                else:
                    self.play_note[note].reward(memory)
                

            memory = np.concatenate([np.zeros((ADDITIONAL_MEMORY)), memory[ADDITIONAL_MEMORY + 13:], song_to_learn.notes[t]])
            t += 1
            memory[t % 8] = 1
        
        if verbose:
            print("Num of Errors: ", errors)
        return errors
    
    def improvise(self, starter, length=128):
        memory = np.zeros((MEMORY_SIZE), dtype=int)
        memory[-STARTER*13:] = starter.notes[:STARTER].flatten()

        result = MySong(starter.notes[:STARTER])
        
        t = STARTER
        memory[t % 8] = 1
        while t < length:    
            output = player.play(memory)
            result.add(output)

            memory = np.concatenate([np.zeros((ADDITIONAL_MEMORY)), memory[ADDITIONAL_MEMORY + 13:], output])
            t += 1
            memory[t % 8] = 1
        result.finish()
        
        return result

## Обучение

Создаём нового игрока и грузим пока кузнечика

In [13]:
player = Player()

In [14]:
kuznechik = Song('basic midi/track (1).mid')

Учим игрока играть кузнечика без ошибок.

In [15]:
errors = -1
while errors != 0:
    errors = player.learn_to_play(kuznechik)
    print(errors)

134
0


Проверим, что всё окей.

In [ ]:
player.improvise(kuznechik).play()

Вот как выглядит его булево правило (в скобках - пороги):

In [17]:
player.play_note[4].print()

0 ∧ not[tb5]E-63G#-61A-59A-57A-53E-51A-49E-47A-45G#-43G#-41G#-37E-35G#-33E-31G#-29A-27A-21B-19B-17B-16B-15B-13C^-11C^-9C^-8C^-7C^-5C^-3B-1( 1 )
 ∧ not[tb4]G#-64G#-60E-58G#-56E-54G#-52A-50A-48A-44E-42A-40E-38A-36G#-34G#-32G#-28E-26G#-24E-22G#-20A-18A-12B-10B-8B-7B-6B-4C^-2( 1 )
 ∧ not[tb3]A-11E-9A-7E-5A-3G#-1( 1 )
 ∧ not[tb2]A-10E-8A-6E-4A-2( 1 )
 ∨ [tb2]A-18E-16A-14E-12A-10G#-8G#-6G#-2( 5.0 )
 ∧ not[tb6]A-46E-44A-42E-40A-38G#-36G#-34G#-30E-28G#-26E-24G#-22A-20A-18A-14E-12A-10E-8A-6G#-4G#-2( 6.0 )
 ∨ [tb2]A-50E-48A-46E-44A-42G#-40G#-38G#-34E-32G#-30E-28G#-26A-24A-22A-18E-16A-14E-12A-10G#-8G#-6G#-2( 13.0 )
 ∧ not[tb2]E-64G#-62E-60G#-58A-56A-54A-50E-48A-46E-44A-42G#-40G#-38G#-34E-32G#-30E-28G#-26A-24A-18B-16B-14B-13B-12B-10C^-8C^-6C^-5C^-4C^-2( 16.0 )
 ∧ not[tb1]A-9E-7A-5E-3A-1( 1 )
 ∧ not[tb0]A-8E-6A-4E-2( 1 )
 ∨ [tb2]A-34E-32A-30E-28A-26G#-24G#-22G#-18E-16G#-14E-12G#-10A-8A-6A-2( 9.0 )
 ∧ not[tb2]E-64A-62E-60A-58G#-56G#-54G#-50E-48G#-46E-44G#-42A-40A-38A-34E-32A-30E-28A-26G#-24G#-22G#-1

Окей, чтобы услышать что-то новое, нужно другое начало. Возьмём его из другой песенки

In [27]:
simple_song = Song('test/track (2).mid')
result = player.improvise(simple_song)

Первый шедевр:

In [28]:
result.play()

Хватаем все песни!

In [39]:
player = Player()

In [40]:
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))

In [41]:
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('test/track (' + str(i) + ').mid'))

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [42]:
import random
random.shuffle(Songs)

In [43]:
for song in Songs:
    errors = player.learn_to_play(song)
    print(errors)

129
79
72
72
48
58
54
68
54
56
56
60
51
36
46
50
49
53
62
45
52
66
63
59
97
65
55
45
57
76
56
76
60
51
61
50
50
55
44
67
73
80
51
64
55
62
53
53
37
68
40
52
41
54
65
66
54
68
53
52
36
54
49
63
69
100
86
98
75


In [44]:
simple_song = Song('test/whomadethis.mid')
result = player.improvise(simple_song)
result.play()

In [45]:
result.save_file("31 transposed songs greedy-linear tb")

In [49]:
simple_song = Song([0, -1, 5, -1, 4, -1, 5, -1])
result = player.improvise(simple_song)
result.play()

In [ ]:
simple_song = Song([0, -1, 5, -1, 4, -1, 7, -1])
result = player.improvise(simple_song)
result.play()

In [38]:
player.play_note[4].print()

0 ∧ not[tb7]C#-63F-61G#-59F-57D#-55F#-53A#-51C-47D#-45F#-43D#-41C#-39F-37G#-35C#-31F-29G#-27F-25D#-23F#-21A#-19F#-17C-15D#-13F#-11D#-9C#-7C#-3( 1 )
 ∧ not[tb5]C#-61F-59G#-57F-55D#-53F#-51A#-49C-45D#-43F#-41D#-39C#-37F-35G#-33C#-29F-27G#-25F-23D#-21F#-19A#-17F#-15C-13D#-11F#-9D#-7C#-5C#-1( 1 )
 ∨ [tb5]C-53D-51E-49F-47G-45C^-41A-37F-35C^-33A-31G-29G-25A-21G-19F-17F-15E-13D-11C-9C-7D-5E-3F-1( 10.0 )
 ∧ not[tb4]C#-60F-58G#-56F-54D#-52F#-50A#-48C-44D#-42F#-40D#-38C#-36F-34G#-32C#-28F-26G#-24F-22D#-20F#-18A#-16F#-14C-12D#-10F#-8D#-6C#-4( 1 )
 ∨ [tb4]F-12G#-8C-4G-4( 5.0 )
 ∨ [tb4]D-12E-10F#-8G-6F#-4A-2( 4.0 )
 ∨ [tb4]G#-12F#-8E-6F#-4G#-2( 4.0 )
 ∨ [tb4]G-12C^-8G-4( 3.0 )
 ∧ not[tb4]C-28D-26E-24F-22G-20C^-16A-12F-10C^-8A-6G-4( 5.0 )
 ∧ not[tb4]G#-28D#-26G#-24D#-22G#-20G-18G-16G-12D#-10G-8D#-6G-4G#-2( 4.0 )
 ∧ not[tb4]G-12D-8G-4( 4.0 )
 ∧ not[tb4]G-12C^-10B-8A-6G-4A-2( 4.0 )
 ∨ [tb4]E-12F#-10G-8A-6B-4( 3.0 )
 ∧ not[tb4]B-20A-16G-14A-12B-10G-8G-6B-4( 4.0 )
 ∧ not[tb4]A-44G-42F#-40G-38A-36D-32A-2

 ∧ not[tb7]A#-63A#-61G-59A#-57A#-55G#-54G-53G#-52F-51E-47C-45C-43C^-41C^-39A#-38G#-37G-36F-35F-33E-31C-29C-27C^-25C^-23A#-22G#-21G-20F-19E-15C-13C-11C^-9C^-7A#-6G#-5G-4F-3F-1( 10.0 )
 ∨ [tb3]F-59D-57F-55D-53F-51D-50F-49G-48A-47F-43D-41F-39D-37F-35G-34F-33A-32D-31F-27D-25F-23D-21F-19D-18F-17G-16A-15F-11D-9F-7D-5F-3G-2F-1( 20.0 )
 ∧ not[tb3]G-64A-63F-59D-57F-55D-53F-51G-50F-49A-48D-47F-43D-41F-39D-37F-35D-34F-33G-32A-31F-27D-25F-23D-21F-19G-18F-17E-16D-15A-11F-9A-7F-5A-3G-2A-1( 21.0 )
 ∨ [tb1]F#-64E-63G#-62C#-61E-57C#-55E-53C#-51E-49C#-48E-47F#-46G#-45E-41C#-39E-37C#-35E-33F#-32E-31D#-30C#-29G#-25E-23G#-21E-19G#-17F#-16G#-15A-14B-13B-11F#-9D#-7F#-5D#-3F#-1( 19.0 )
 ∧ not[tb0]C#-8F-6G#-4F-2( 1 )
 ∨ [tb0]C-64G-64E-60F-56G#-52C-48G-48E-44F-40G#-40C^-36D-32A#-32G-28C-24G#-24F-20E-16G-16F-8G#-8C^-4( 10.0 )
 ∧ not[tb0]D-64A#-64G-60C-56G#-56F-52E-48G-48F-40G#-40C^-36E-32A#-32G-28C#-24G#-24F-20C-16G-16F-8G#-8C^-4( 14.0 )
 ∨ [tb0]F-56G#-52C-48G-48E-44F-40G#-36C-32G-32E-28F-24G#-24C^-20D-16A#-16G-

 ∨ [tb0]G-64E-60F-56E-54D-52F-50E-48C-44G-40C^-36G-32E-28F-24E-22D-20F-18E-16C-12F-8D-4D-2( 12.0 )
 ∨ [tb0]G-24C^-20G-16E-12F-8E-6D-4F-2( 5.0 )
 ∧ not[tb0]C-32F-28G-24G#-20G-16E-12F-8( 6.0 )
 ∨ [tb2]G-18C^-14G-10E-6F-2( 4.0 )
 ∧ not[tb2]F-18G#-14C-10G-10E-6F-2( 5.0 )
 ∧ not[tb2]G-50E-46G-42E-38D-34E-32F-30D-28E-26C-22G-18E-14G-10E-6D-2( 11.0 )
 ∧ not[tb6]A-62G-58F-56E-54D-50C-46E-38F-36G-34G-30A-28G-26E-22F-20G-18G-14A-12G-10E-6G-4F-2( 6.0 )
 ∧ not[tb2]C-26F-22G-18G#-14G-10E-6F-2( 6.0 )
 ∧ not[tb2]D#-26F-24G-22G#-20G-18A#-16F-14G-10A#-8F-6F-2( 5.0 )
 ∧ not[tb2]F-34F-32F-30A-28C^-26C^-24C^-22A-20G-18G-16C^-14A#-12A-10F-6F-2( 6.0 )
 ∨ [tb2]F-64E-62D-60C-58D-56E-54G-50C^-48B-46A-44G-42A-40G-38F-36E-34F-32E-30D-28C-26E-23C-22G-18G-16C^-14B-12A-10G-8E-6G-4C-2( 16.0 )
 ∧ not[tb2]F-26G-22A-18C^-14A-12G-10A-6G-4F-2( 5.0 )
 ∨ [tb0]A-40G-36F-34G-32A-30F-28F-26A-24G-20F-18G-16A-14F-12F-8E-4D-2( 8.0 )
 ∧ not[tb0]A#-64C^-62G#-60G#-58C^-56A#-52G#-50A#-48C^-46G#-44G#-40G-36F-34G-32G#-30F-28F-26G#-24G

 ∨ [tb6]A#-64A-62F-58F-54F-52F-50A-48C^-46C^-44C^-42A-40G-38G-36C^-34A#-32A-30F-26A-22G-20F-18E-16D-14E-12F-10G-8A-6G-4F-2( 18.0 )
 ∧ not[tb0]C-8D-6E-4F-2( 6.0 )
 ∧ not[tb0]B-64E-60F#-56A-54G-52F#-50E-48E-40F#-38G-36A-34B-32E-28B-24A-22G-20F#-18E-16C-8D-6E-4F#-2( 6.0 )
 ∨ [tb4]B-64A-62G#-60E-56G#-52F#-50E-48D#-46C#-44D#-42E-40F#-38G#-36F#-34E-32D#-30C#-28C#-24G#-20F#-18E-16D#-14C#-12D#-10E-8F#-6G#-4F#-2( 11.0 )
 ∨ [tb4]B-64A-62G#-60E-56E-52E-50E-48G#-46B-44B-42B-40G#-38F#-36F#-34B-32A-30G#-28E-24G#-20F#-18E-16D#-14C#-12D#-10E-8F#-6G#-4F#-2( 18.0 )
 ∨ [tb4]B-64G#-62F#-60F#-58B-56A-54G#-52E-48E-44E-42E-40G#-38B-36B-34B-32G#-30F#-28F#-26B-24A-22G#-20E-16G#-12F#-10E-8D#-6C#-4D#-2( 17.0 )
 ∨ [tb4]G-52E-48G-44E-40D-36E-34F-32D-30E-28C-24G-20E-16G-12E-8D-4F-2( 11.0 )
 ∨ [tb0]C^-64G-60A-56G-52F-50E-48D-44C-40C^-32G-28A-24G-20F-18E-16D-12C-8( 10.0 )
 ∨ [tb0]E-56F#-54G-52F#-50G-48E-44G-40A-38B-36A-34B-32B-24A-22G-20F#-18G-16E-12A-8G-6F#-4G-2( 10.0 )
 ∧ not[tb2]F#-64F#-62F#-58D-56F#-54D-52F#-50G-